# 量子コンピューターの中を覗いてみよう！
```{admonition} 注意点
:class: warning

第二章を理解するためには量子力学の知識が必要です．詳しく知りたい方は量子力学の入門書を最初に読むことをお勧めします．(もちろん将来的には前提知識によらず説明できるような教材の改定を目指しています．)

```
## なぜ量子コンピューターの中身に興味が湧くか
第一章の量子情報のセクションでは$0,1$を重ね合わせにすることができると話しました．しかし，$0,1$自体は単なる情報の単位でしかありません．一方，量子力学的な重ね合わせの現象は電子などのミクロなスケールの世界で表されます．つまり，量子ビットは物理的に実装されており，何もただ単なる単位や存在しない何かではないのです．また，量子ビット以外にも，量子アルゴリズムで出てきたゲートとは一体何であるのか，気になっている方もいるのでないでしょうか?

```{image} ../../images/物理的実装.png
:width: 400px
:align: center
```
このような疑問が生じることはごく自然なことです．というのも**情報は物理的**だからです．我々がいつも使っているコンピューターで行われるすべての処理やメモリなどの全ての情報は空想の何かとかではなく，物理的な何かによって実現されています．物理的に操作して保存したものを測定をして我々はコンピューターを扱えるのです．量子コンピューターももちろん，量子ビットやゲートも物理的に実現されています．<br>
完全に量子コンピュータの実装を知っている必要はありませんが，そもそも概念ですら理解しずらい量子コンピュータを，具体的な物理的な物を出さずに情報科学的な抽象的な段階だけで説明されてもより混乱するだけと思っています．<br>
そこで，量子計算をより理解するために，この章では，**量子ビットとは物理的に何か？**，**量子ゲートは何を行っているのか**が物理的に想像でき，量子計算を行う上で実感を持つことを目標にしています．しかし，ただ単に実装を知ることは量子計算の理解が深まるだけではなく，面白い世界が広がります．実装を知っていると役に立つ一例です．

```{admonition} 実装を知っていると役に立つ例1
:class: dropdown
量子計算では$|0\rangle,|1\rangle$が基本単位になっていると説明しました．しかし実際の超伝導量子ビットでは$|0\rangle,|1\rangle$以外にも物理的に$|2\rangle,|3\rangle,\cdots$と他の状態も存在します．通常ではこれらは計算には使わずこれらを$|0\rangle,|1\rangle$からよく分離をしなければならのですが，これらを利用して量子計算ができることを増やしたり計算効率を上げるアルゴリズムの研究が行われいます{cite:p}`Wang_2020`．これらの量子ビットはQuditと呼ばれております．
```

```{admonition} 実装を知っていると役に立つ例2
:class: dropdown
大雑把なエラー率が分かるようになります．例として以下の例を考えましょう．<br>
量子状態トモグラフィーを説明した際に，$Y$測定を行うときは，$S_X$ゲートを通して測定を行いました．しかし，Qiskitのモジュールである[Qiskit Experiment](https://qiskit-extensions.github.io/qiskit-experiments/stubs/qiskit_experiments.library.tomography.basis.PauliMeasurementBasis.html#)ではY測定を行う場合は$S^{\dagger}\to H$という形で二つのゲートを通しています．<br>
実際にはそれぞれの場合でたしかに$Y$測定を行うことができます．直感的に考えればゲートのエラー率はゲートを二回通している$S^{\dagger}\to H$の方が高くなると考えるかもしれません．しかし，それぞれのゲート操作はでエラー率は等しくなります． それぞれのゲートを$X_{\theta_x}, Z_{\theta_z}$ゲートで分解すると，
\begin{align}
H\cdot S^{\dagger}  &=  \left( Z_{\pi/2}\cdot X_{\pi/2}\cdot Z_{\pi/2} \right) \cdot Z_{-\pi/2} =Z_{\pi/2}\cdot  X_{\pi/2}\\
S_X &= e^{i\pi/4} X_{\pi/2}
\end{align}
となっています．(今回は1量子ビットを考えているためグローバル位相の部分は関係ありません．)<br>
実は，後で説明をするように$Z_{\theta_z}$は仮想的に実装されており$Z_{\theta_z}$を実現するためにパルスなどを当てておらずエラー率という概念が存在しません．以上の理由からエラー率に関与するのは，$X_{\pi/2}$ゲートのみになっており，それぞれでエラー率は等しいと言えます．
```

## 超伝導量子コンピュータ
量子コンピュータには様々な方式が提案されています．光を利用したものから，イオンをトラップして利用するものまで多岐に渡っています．もちろん，
各量子コンピュータにそれぞれメリット，デメリットがあり量子コンピュータの先行きは不透明です．ただ，2024年現在，日本国産初号機の量子コンピュータや，個人で無料で使用できる(もちろん制限はあります)IBMの量子コンピュータが超伝導を利用した量子コンピュータであるため，この教材では超伝導量子コンピュータに関して取り扱います．

### 全体の構造
超伝導量子コンピュータは以下のような大きなシャンデリアのような形をしております．

```{figure} ../../images/IBM_SystemOne_Andrew_Lindemann_2.jpg
:width: 400px
:align: center

IBMの量子コンピュータの写真．[IBMのホームページ](https://newsroom.ibm.com/media-quantum-innovation?keywords=quantum&l=100#gallery_gallery_0:21596)より引用
```
このシャンデリアのような量子コンピュータは下に行けば行くほど温度が下がり$0$K付近まで冷えています．そして，$0$Kまで冷えている箇所を見ると，以下のような実際の量子の演算を行うプロセッサーが出てきます．
```{figure} ../../images/POWER10_06.jpg
:width: 400px
:align: center

IBMの量子プロセッサーの写真．[IBMのホームページ](https://newsroom.ibm.com/media-quantum-innovation?keywords=quantum&l=100#gallery_gallery_0:21676)より引用
```
このプロセッサーの中には量子ビットが集積化されており，そこで実際の演算を行うことができます．この超伝導量子コンピュータで使用されている量子ビットのことを**超伝導量子ビット**と言います．

### 超伝導量子ビットの様々な種類

ちょっと頭が混乱するかもしれませんが，実は超伝導量子ビットにも様々な手法があるんです... 様々な手法のうち，以下の二つ紹介します．

````{card}
````{tab-set}
```{tab-item} 周波数固定型トランズモン量子ビット
主にIBMで開発が進められている量子ビットです．パルスのみで量子ビットを操作します．
```

```{tab-item} 周波数可変型トランズモン量子ビット
主にGoogleで開発が進められている量子ビットであり，$|0\rangle$と$|1\rangle$の間にあるエネルギー間隔を人工的に操作できます．そのため，パルスに加えて，外部磁場を加える必要があります．
```
````

次の章からはこれらのうち，IBMなどの企業で実際に使われている**周波数固定型トランズモン量子ビット**に関して焦点を当てて話していきます．

```{note}
それぞれに利点と欠点がありそもそもどの方式の量子コンピューターが今後広く使われていくのか，もし仮に超伝導量子ビットであれば，超伝導量子ビットのどの手法が用いられるのかなどまだまだ研究段階であり分かっていないことは多いです．
```